# Castro et al. (2019) Data Processing and Analysis

## Setup

In [2]:
import os
import sys
import glob
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
import re
import pickle
from collections import Counter
from functools import reduce

import pandas as pd
import scipy
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl

In [4]:
from cvtk.cvtk import TemporalFreqs, TiledTemporalFreqs
from cvtk.cov import stack_temporal_covariances
import cvtk.variant_files as vf
from cvtk.gintervals import GenomicIntervals
from cvtk.pca import FreqPCA
from cvtk.plots import rep_plot_pca, correction_diagnostic_plot
from cvtk.utils import integerize
from cvtk.utils import extract_empirical_nulls_diagonals, extract_temporal_cov_diagonals
from cvtk.cov import stack_replicate_covariances, stack_temporal_covs_by_group
from cvtk.variant_files import VCFFile

In [5]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
mpl.rcParams['figure.figsize'] = (8.0, 4.0)
mpl.rcParams['figure.dpi'] = 200

# Varianta Data Loading

### Load in VCF data

In [7]:
vcf = VCFFile('../data/castro_et_al_2019/beagle_genMap.all.impute.vcf.gz')

reading file '../data/castro_et_al_2019/beagle_genMap.all.impute.vcf.gz'...
file '../data/castro_et_al_2019/beagle_genMap.all.impute.vcf.gz' loaded.
total time to load VCF file: 9.346558225154876 mins.


Remove fixed sites — those that are not polymorphic in any samples / timepoints. These just needlessly shrink the covariance towards zero.

### Sample Data

The samples names to line/generation mapping was not in a simple text file, but I found the relevant information in the vcftools commands that were at the beginning of the file `Longshanks_F0F17.summary_stats.tar.gz`. From this I created `samples.txt`, which is read in and parsed below.

In [8]:
samples = pd.read_csv("../data/castro_et_al_2019/samples.txt", header=None, names = ('line', 'individual'))
sample_map = {k:v for k, v in zip(samples['individual'], samples['line'])}

subpop_indices = defaultdict(list)
for i, k in enumerate(vcf.samples):
    subpop_indices[sample_map[k.decode()]].append(i)

From this, we can map the `vcf.geno_mat` table to subpopulation counts. 

In [9]:
counts_mat = vcf.count_alleles_subpops(subpop_indices)

/home/vsb/miniconda3/envs/cvtk/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
vcf.subpops

dict_keys(['Ctrl_F17', 'LS1_F17', 'LS2_F17', 'Ctrl_F0', 'LS1_F0', 'LS2_F0'])

Now we count the number of diploids in each sample.

In [11]:
ndiploids = [Counter(sample_map.values())[k] for k in vcf.subpops]

In [12]:
def parse_samples(x):
    line, gen = x.split('_')
    return (line, gen[1:])

design = [parse_samples(x) for x in vcf.subpops]

In [13]:
freq_mat_all = vcf.calc_freqs()

In [14]:
print("number of loci: ", freq_mat_all.shape[1])

number of loci:  31944210


With the frequencies calculated, now we filter out all non-segregating sites.

In [15]:
vcf.remove_fixed()
freq_mat = vcf.calc_freqs()
print("number of loci: ", freq_mat.shape[1])
print("loci not segregating removed: ", freq_mat_all.shape[1] - freq_mat.shape[1])

number of loci:  8162172
loci not segregating removed:  23782038


In [16]:
gi = vcf.build_gintervals()

## Replicate Covariance Analysis

In [17]:
tile_width = 10e6
gi.infer_seqlens()
tiles = GenomicIntervals.from_tiles(gi.seqlens, width=tile_width)

In [18]:
d = TiledTemporalFreqs(tiles, freqs=freq_mat, depths=vcf.N, diploids=ndiploids, samples=design, gintervals=gi)

In [87]:
d.freqs.shape

(3, 2, 8162172)

In [19]:
d.samples

[('Ctrl', '0'),
 ('Ctrl', '17'),
 ('LS1', '0'),
 ('LS1', '17'),
 ('LS2', '0'),
 ('LS2', '17')]

In [20]:
autosomes = list(set(gi.intervals.keys()) - set('chrX'))

In [21]:
covs_cis = d.bootstrap_cov(B=5000, keep_seqids=autosomes, average_replicates=False, progress_bar=True)

In [22]:
covs_cis

array([[[ 0.07650138, -0.0200637 , -0.01453671],
        [-0.0200637 ,  0.1076956 ,  0.0130842 ],
        [-0.01453671,  0.0130842 ,  0.12422544]],

       [[ 0.10294168, -0.00699108, -0.00235625],
        [-0.00699108,  0.13691369,  0.02631757],
        [-0.00235625,  0.02631757,  0.16493685]],

       [[ 0.11889015,  0.00682645,  0.01455945],
        [ 0.00682645,  0.15482518,  0.03951696],
        [ 0.01455945,  0.03951696,  0.18938609]]])

In [23]:
with open('../data/castro_et_al_2019/covs_bootstrap_10e6.npy', 'wb') as f:
    np.save(f, covs_cis)

### Bootstrap the Convergence Correlation

In [75]:
convergence_corr_LSs = d.bootstrap_convergence_corr(B=5000, progress_bar=True, subset=(1, 2))
convergence_corr_LSs

array([[[[0.07301283]]],


       [[[0.17513376]]],


       [[[0.25480595]]]])

In [76]:
convergence_corr_LS1_Ctrl = d.bootstrap_convergence_corr(B=5000, progress_bar=True, subset=(0, 1))
convergence_corr_LS1_Ctrl

array([[[[-0.16817056]]],


       [[[-0.05888775]]],


       [[[ 0.0723813 ]]]])

In [77]:
convergence_corr_LS2_Ctrl = d.bootstrap_convergence_corr(B=5000, progress_bar=True, subset=(0, 2))
convergence_corr_LS2_Ctrl

array([[[[-0.12909668]]],


       [[[-0.01808309]]],


       [[[ 0.11439289]]]])

### Combining and saving convergence correlations

In [78]:
all_conv_corrs = np.stack((convergence_corr_LS1_Ctrl, 
                           convergence_corr_LS2_Ctrl, 
                           convergence_corr_LSs,)).squeeze().T
all_conv_corrs

array([[-0.16817056, -0.12909668,  0.07301283],
       [-0.05888775, -0.01808309,  0.17513376],
       [ 0.0723813 ,  0.11439289,  0.25480595]])

In [79]:
with open('../data/castro_et_al_2019/all_conv_corrs.npy', 'wb') as f:
    np.save(f, all_conv_corrs)

### Partitioning Variance into Shared and Unique Contributions

In [22]:
# this is specific to the Castro study
def unique_shared(cov):
    sel = (cov[1,1]+cov[2,2])/2 - cov[0, 0]
    shared = cov[1,2]
    return sel - shared, shared

In [23]:
# cols/rows are Ctl, LS1, LS2
gw_covs = d.calc_cov()
gw_covs

array([[ 0.10294167, -0.00699108, -0.00235626],
       [-0.00699108,  0.13691369,  0.02631757],
       [-0.00235626,  0.02631757,  0.16493685]])

A quick estimate of a G analog using replicate covariances:

In [24]:
gw_covs[1,2]/((gw_covs[1,1] + gw_covs[2,2])/2)

0.1743748260253365

In [25]:
uni, shared = unique_shared(gw_covs)
print('unique:', uni)
print('shared:', shared)

unique: 0.021666025514262117
shared: 0.026317567154741282


In [26]:
from cvtk.bootstrap import bootstrap_ci, block_bootstrap_ratio_averages

In [27]:
windowed_covs = d.calc_cov_by_tile()                            

In [28]:
windowed_stats = np.array([unique_shared(win) for win in windowed_covs])

Unique and shared contributions to G bootstrap:

In [34]:
B = 5000
windowed_stats = np.array([unique_shared(win) for win in windowed_covs])
vars = np.array([(win[1,1] + win[2,2])/2 for win in windowed_covs])
uni_straps, shared_straps, total_sel_straps = [], [], []
for b in np.arange(B):
    bidx = np.random.randint(0, len(windowed_covs), len(windowed_covs))
    u, s = windowed_stats[bidx, :].mean(axis=0)
    var_ls = vars[bidx].mean()
    b =(u + s) / var_ls
    uni_straps.append(u/var_ls)
    shared_straps.append(s/var_ls)
    total_sel_straps.append(b)
    
ls_var = (gw_covs[1,1] + gw_covs[2,2])/2
print('unique:', (bootstrap_ci(uni/ls_var, uni_straps) * 100).round(1))
print('shared:', (bootstrap_ci(shared/ls_var, shared_straps) * 100).round(1))
print('total sel:', (bootstrap_ci((uni + shared)/ls_var, total_sel_straps) * 100).round(1))

unique: [ 3.3 14.4 33.3]
shared: [ 8.5 17.4 23.4]
total sel: [20.6 31.8 47.8]


### Sign test of variance magnitude

In [102]:
B = 5000

windowed_covs = np.array(windowed_covs)

straps1, straps2 = [], []
for b in np.arange(B):
    bidx = np.random.randint(0, len(windowed_covs), len(windowed_covs))
    d1 = np.mean(windowed_covs[bidx, 1, 1] - windowed_covs[bidx, 0, 0])
    d2 = np.mean(windowed_covs[bidx, 2, 2] - windowed_covs[bidx, 0, 0])
    straps1.append(d1)
    straps2.append(d2)
    shared_straps.append(s)
    
print('1:', bootstrap_ci(gw_covs[1, 1] - gw_covs[0, 0], straps1))
print('2:', bootstrap_ci(gw_covs[2, 2] - gw_covs[0, 0], straps2))

1: [0.01127199 0.03397201 0.05654474]
2: [0.03907203 0.06199516 0.10146228]


## Analysis Excluding Chromosomes 5 and 10

In [33]:
autosomes_sans_chr5_and_chr10 = [chr for chr in autosomes if chr not in ('chr5', 'chr10')]

In [40]:
covs_sans_chr5_and_chr10_cis = d.bootstrap_cov(B=5000, keep_seqids=autosomes_sans_chr5_and_chr10, 
                                              average_replicates=False, progress_bar=True)

/home/vinceb/projects/cvtk/cvtk/cov.py:283: RuntimeWarning: Mean of empty slice
  mean_hets = np.nanmean(hets, axis=freqs.ndim-1)
/home/vinceb/miniconda3/envs/basesci/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/vinceb/projects/cvtk/cvtk/cov.py:309: RuntimeWarning: Degrees of freedom <= 0 for slice
  cov = np.cov(deltas, bias=True)
/home/vinceb/miniconda3/envs/basesci/lib/python3.6/site-packages/numpy/lib/function_base.py:2451: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/home/vinceb/projects/cvtk/cvtk/cov.py:338: RuntimeWarning: Mean of empty slice
  ave_bias += np.nanmean(0.5 * hets * (diploid_correction + depth_correction), axis=2)


In [41]:
covs_sans_chr5_and_chr10_cis

array([[[ 0.07912359, -0.02006789, -0.01685076],
        [-0.02006789,  0.10007415,  0.01004145],
        [-0.01685076,  0.01004145,  0.12769844]],

       [[ 0.10678867, -0.00736898, -0.00733722],
        [-0.00736898,  0.13543516,  0.02544975],
        [-0.00733722,  0.02544975,  0.14899192]],

       [[ 0.12436063,  0.01076925,  0.00994585],
        [ 0.01076925,  0.15168951,  0.03887876],
        [ 0.00994585,  0.03887876,  0.17238601]]])

In [42]:
with open('../data/castro_et_al_2019/covs_sans_chr5_and_chr10_bootstrap_10e6.npy', 'wb') as f:
    np.save(f, covs_sans_chr5_and_chr10_cis)

### Convergence Correlations without Chr 5 and Chr 10

In [80]:
corr_LSs_sans_chr5_and_chr10 = d.bootstrap_convergence_corr(B=5000, 
                                                    keep_seqids=autosomes_sans_chr5_and_chr10, 
                                                    progress_bar=True,
                                                    subset=(1, 2))
corr_LSs_sans_chr5_and_chr10

array([[[[0.06286245]]],


       [[[0.17916208]]],


       [[[0.2697911 ]]]])

In [81]:
corr_LS1_Ctrl_sans_chr5_and_chr10 = d.bootstrap_convergence_corr(B=5000, 
                                                    keep_seqids=autosomes_sans_chr5_and_chr10, 
                                                    progress_bar=True,
                                                    subset=(0, 1))
corr_LS1_Ctrl_sans_chr5_and_chr10

array([[[[-0.16853583]]],


       [[[-0.06127422]]],


       [[[ 0.09919115]]]])

In [82]:
corr_LS2_Ctrl_sans_chr5_and_chr10 = d.bootstrap_convergence_corr(B=5000, 
                                                    keep_seqids=autosomes_sans_chr5_and_chr10, 
                                                    progress_bar=True,
                                                    subset=(0, 2))
corr_LS2_Ctrl_sans_chr5_and_chr10

array([[[[-0.13101864]]],


       [[[-0.05816867]]],


       [[[ 0.09967332]]]])

In [83]:
all_conv_corrs_sans_chr5_and_chr10 = np.stack((corr_LS1_Ctrl_sans_chr5_and_chr10, 
                                               corr_LS2_Ctrl_sans_chr5_and_chr10,
                                               corr_LSs_sans_chr5_and_chr10)).squeeze().T
all_conv_corrs_sans_chr5_and_chr10

array([[-0.16853583, -0.13101864,  0.06286245],
       [-0.06127422, -0.05816867,  0.17916208],
       [ 0.09919115,  0.09967332,  0.2697911 ]])

In [84]:
with open('../data/castro_et_al_2019/conv_corrs_sans_chr5_and_chr10.npy', 'wb') as f:
    np.save(f, all_conv_corrs_sans_chr5_and_chr10)

In [85]:
all_conv_corrs_sans_chr5_and_chr10

array([[-0.16853583, -0.13101864,  0.06286245],
       [-0.06127422, -0.05816867,  0.17916208],
       [ 0.09919115,  0.09967332,  0.2697911 ]])

In [86]:
all_conv_corrs

array([[-0.16817056, -0.12909668,  0.07301283],
       [-0.05888775, -0.01808309,  0.17513376],
       [ 0.0723813 ,  0.11439289,  0.25480595]])